## Gebruikersprofielen

In [ ]:
import json
from geopy.distance import geodesic
import numpy as np
import datetime

file_path = '../Data/Volledigedata/Loopdata/runs_with_speed.json'
user_profiles = {}

def calculate_distance(coords):
    return sum(geodesic(coords[i], coords[i+1]).kilometers for i in range(len(coords) - 1))

def calculate_average_location(coords):
    if not coords:
        return None
    latitudes, longitudes = zip(*coords)
    return np.mean(latitudes), np.mean(longitudes)

def calculate_condition_value(activity):
    if 'heart_rate' in activity and 'speed' in activity and 'timestamp' in activity:
        coords = list(zip(activity['latitude'], activity['longitude']))

        if not coords:
            print(f"Activiteit {activity['id']} overgeslagen: geen geldige coördinaten gevonden.")
            return None

        avg_heart_rate = np.mean(activity['heart_rate'])
        avg_speed = np.mean(activity['speed'])

        if avg_heart_rate == 0:
            print(f"Activiteit {activity['id']} overgeslagen: gemiddelde hartslag is nul.")
            return None

        distance = calculate_distance(coords)
        start_time = datetime.datetime.fromtimestamp(activity['timestamp'][0])
        end_time = datetime.datetime.fromtimestamp(activity['timestamp'][-1])
        duration = (end_time - start_time).total_seconds() / 3600

        if duration == 0:
            print(f"Activiteit {activity['id']} overgeslagen: duur is nul.")
            return None

        # Aangepaste formule voor condition_value zonder hartslagvariabiliteit
        condition_value = (avg_speed * distance) / (avg_heart_rate * duration)
        return condition_value
    else:
        return None

# Process data
with open(file_path, 'r') as file:
    for line in file:
        try:
            activity = json.loads(line)
            user_id = activity.get('userId')

            if user_id:
                if user_id not in user_profiles:
                    user_profiles[user_id] = {
                        'userId': user_id,
                        'activity_ids': [],
                        'speeds': [],
                        'distances': [],
                        'average_heartrates': [],
                        'altitude_changes': [],
                        'locations': [],
                        'condition_values': [],
                    }

                coords = list(zip(activity['latitude'], activity['longitude']))
                altitude = activity.get('altitude', [])
                altitude_change = max(altitude) - min(altitude) if altitude else 0

                user_profiles[user_id]['activity_ids'].append(activity.get('id', 'unknown'))
                user_profiles[user_id]['speeds'].append(np.mean(activity.get('speed', [])))
                user_profiles[user_id]['distances'].append(calculate_distance(coords))
                user_profiles[user_id]['average_heartrates'].append(np.mean(activity.get('heart_rate', [])))
                user_profiles[user_id]['altitude_changes'].append(altitude_change)
                user_profiles[user_id]['locations'].append(calculate_average_location(coords))
                condition_value = calculate_condition_value(activity)
                if condition_value is not None:
                    user_profiles[user_id]['condition_values'].append(condition_value)
                
        except json.JSONDecodeError as e:
            print(f"Fout bij het lezen van de regel: {e}")

# Berekenen van gemiddelde locatie voor elk profiel
for user_id, profile in user_profiles.items():
    locations = profile['locations']
    if locations:
        average_location = calculate_average_location(locations)
        profile['average_location'] = average_location
    else:
        profile['average_location'] = (None, None)

    # Calculating averages and standard deviations
    for key in ['speeds', 'distances', 'average_heartrates', 'altitude_changes']:
        profile[f'average_{key}'] = np.mean(profile[key])
        profile[f'std_{key}'] = np.std(profile[key])

    # Calculating weights
    if profile['condition_values']:
        profile['std_condition_values'] = np.std(profile['condition_values'])
    else:
        profile['std_condition_values'] = 0
    
    std_distance = profile['std_distances']
    std_altitude_change = profile['std_altitude_changes']
    std_condition_value = profile['std_condition_values'] 
    total_std = (1/(std_condition_value+0.0001)) + (1/(std_distance+0.0001)) + (1/(std_altitude_change+0.0001))

    profile['weight_std_condition_value'] = (1/(std_condition_value+0.0001)) / total_std
    profile['weight_std_distance'] = (1/(std_distance+0.0001)) / total_std
    profile['weight_std_altitude_change'] = (1/(std_altitude_change+0.0001)) / total_std

    if profile['condition_values']:
        profile['average_condition_value'] = np.mean(profile['condition_values'])
    else:
        profile['average_condition_value'] = None

# Save to file
output_file_path = '../Data/Loopdata/kleinebestanden/run_gebruikersprofielen.json'
with open(output_file_path, 'w') as outfile:
    for profile in user_profiles.values():
        json.dump(profile, outfile)
        outfile.write('\n')


## Routes

In [ ]:
import json
from geopy.distance import geodesic

# Lees het oorspronkelijke bestand met JSON-objecten
input_file_path = '../Data/Volledigedata/Loopdata/runs_with_speed.json'
output_file_path = '../Data/Loopdata/kleinebestanden/looproutes.json'

# Lijst om de relevante velden voor routes bij te houden
relevant_fields = ['latitude', 'longitude', 'altitude', 'sport', 'id', 'heart_rate', 'timestamp', 'userid', 'speed']

routes = []

def calculate_distance(coords):
    return sum(geodesic(coords[i], coords[i+1]).kilometers for i in range(len(coords) - 1))

def calculate_altitude_change(altitudes):
    return max(altitudes) - min(altitudes) if altitudes else 0

with open(input_file_path, 'r') as infile:
    for line in infile:
        activity = json.loads(line)
        route_data = {field: activity[field] for field in relevant_fields if field in activity}

        # Bereken en voeg afstand en hoogteverandering toe
        if 'latitude' in activity and 'longitude' in activity:
            coords = list(zip(activity['latitude'], activity['longitude']))
            route_data['distance'] = calculate_distance(coords)
        
        if 'altitude' in activity:
            route_data['altitude_change'] = calculate_altitude_change(activity['altitude'])

        routes.append(route_data)

# Stap 2: Schrijf de routes naar een apart JSON-bestand met een nieuwe lijn na elk JSON-object
with open(output_file_path, 'w') as outfile:
    for route in routes:
        json.dump(route, outfile)
        outfile.write('\n')  # Voeg een nieuwe lijn toe na elk JSON-object

print(f"{len(routes)} routes zijn opgeslagen in {output_file_path}.")


## Koppelen routes/activiteiten met gebruikersinfo

In [ ]:
import json
import os

def read_json_lines(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Het bestand {file_path} bestaat niet.")
    with open(file_path, 'r') as file:
        return [json.loads(line) for line in file]

def write_json_lines(file_path, data):
    with open(file_path, 'w') as file:
        for item in data:
            file.write(json.dumps(item) + '\n')

def koppel_activiteiten_met_gebruikers(activiteiten_bestand, gebruikersprofielen_bestand):
    gebruikersprofielen = read_json_lines(gebruikersprofielen_bestand)
    fietsactiviteiten = read_json_lines(activiteiten_bestand)

    activiteit_user_mapping = {}
    for gebruiker in gebruikersprofielen:
        for activiteit_id in gebruiker['activity_ids']:
            activiteit_user_mapping[activiteit_id] = {
                'userId': gebruiker['userId'],
                'average_condition_value': gebruiker['average_condition_value']
            }

    for activiteit in fietsactiviteiten:
        activiteit_id = activiteit['id']
        if activiteit_id in activiteit_user_mapping:
            activiteit.update(activiteit_user_mapping[activiteit_id])
    
    return fietsactiviteiten

activiteiten_bestand = '../Data/Loopdata/kleinebestanden/looproutes.json'
gebruikersprofielen_bestand = '../Data/Loopdata/kleinebestanden/run_gebruikersprofielen.json'

try:
    aangepaste_fietsactiviteiten = koppel_activiteiten_met_gebruikers(activiteiten_bestand, gebruikersprofielen_bestand)
    write_json_lines('../Bestanden_zonder_snelheid/bikedata_volledig/routes_volledig.json', aangepaste_fietsactiviteiten)
    print(aangepaste_fietsactiviteiten[:5])
except FileNotFoundError as e:
    print(e)
except Exception as e:
    print(f"Er is een onverwachte fout opgetreden: {e}")
